In [6]:
import yaml
import os
import numpy as np
import copy

In [7]:
base_config={
                   #'model_name': ,
                   #'model_path': ,
                   #'device': ,
                   'class_groups': [[2*i,2*i+1] for i in range(5)],
                   #'dataset_name':'MNIST' or 'CIFAR',
                   #'dataset_type': 'std', 'group', 'corrupt' or 'mark',
                   #'data_root':,
                   'batch_size': 32,
                   #'save_dir': ,
                   'validation_size': 2000,
                   'accuracy': True,
                   'start_file': 0,
                   'num_files': 20
}
project_root="C:/Users/weckbecker/DualView-wip"
dataset_root="C:/Users/weckbecker/DualView-wip/src/datasets"
data_root="C:/Users/weckbecker/DualView-wip/src/datasets"

In [8]:
def create_config(config, config_name):
    config['data_root'] = dataset_root
    config['device'] = 'cuda'
    if config['dataset_type'] in ["switched_one_file", "std_one_file"]:
        config['model_path']=f"{project_root}/checkpoints/{config['dataset_name']}/std/{config['model_name']}_std/{config['dataset_name']}_{config['model_name']}"
    else:
        config['model_path']=f"{project_root}/checkpoints/{config['dataset_name']}/{config['dataset_type']}/{config['model_name']}_{config['dataset_type']}/{config['dataset_name']}_{config['model_name']}"
    config['save_dir_explainer'] = f"{project_root}/explanations/{config['dataset_name']}/{config['dataset_type']}/{config['model_name']}_{config['dataset_type']}/{config['xai_method']}"
    config['save_dir_results'] = f"{project_root}/surrogate_check/{config['dataset_name']}/{config['dataset_type']}/{config['model_name']}_{config['dataset_type']}/{config['xai_method']}"
    if config['xai_method'] == 'dualview_model_labels':
        config['xai_method'] = 'dualview'
    path = f"surrogate_check/{config['dataset_name']}"
    os.makedirs(path, exist_ok=True)
    if config['xai_method'] in ["dualview", 'dualview_model_labels']:
        config['save_dir_explainer']=f"{config['save_dir_explainer']}_{config['C']}"
        config['save_dir_results']=f"{config['save_dir_results']}_{config['C']}"
    with open(f"{path}/{config_name}.yaml", 'w') as outfile:
        yaml.dump(config, outfile, default_flow_style=False)


In [9]:
names=[('MNIST','basic_conv'),('CIFAR','resnet')]
xai_methods=['dualview','representer', 'rp_similarity', 'similarity', 'dualview_model_labels']
dataset_types=['std', 'group', 'mark', 'corrupt']
C_list=['0.00001', '0.0001', '0.001', '0.01', '0.1', '1', '10', '100']
batch_size=32

In [10]:
from copy import deepcopy

base_config['num_files']=500
for dsname,model_name in names:
    base_config['model_name']=model_name
    base_config['dataset_name']=dsname
    base_config['batch_size']=batch_size 
    for dstype in dataset_types:
        if dstype=='group':
            base_config['num_classes']=len(base_config['class_groups'])
        else:
            base_config['num_classes']=10
        base_config['dataset_type']=dstype
        if dstype in ['switched_one_file', 'std_one_file']:
            base_config['num_batches_per_file'] = 100000
        else:
            base_config['num_batches_per_file'] = 100
        for xai_method in xai_methods:
            base_config['xai_method']=xai_method  
            if xai_method in ['dualview', 'dualview_model_labels']:
                for C in C_list:
                    base_config['C'] = C
                    create_config(deepcopy(base_config), f"{model_name}_{dstype}_{xai_method}_{C}")
            else:
                create_config(deepcopy(base_config), f"{model_name}_{dstype}_{xai_method}")